#### Stage 4
# Origin Destination Data - Spatial Overview - Anomalies
with folium maps

#### Generating Flow Maps Showing Anomalies in OD Data
This notebook contains the code for the generation of an **interactive flow map** from [anomODdf.csv](https://www.mcloud.de/downloads/ingrid-group_ige-iplug-mcloud/9A101FEC-3502-495A-9D93-BD8329A9D8AC/anomODdf.csv).
The plotting function is similar to that at [stage two](https://github.com/klavere/mCLOUDxMNDvis/blob/main/2_flowData_spatialOverview_anomalies.ipynb) but incorporating colouring of the start- and end-cells as at [stage three](https://github.com/klavere/mCLOUDxMNDvis/blob/main/3_originDestinationData_spatialOverview.ipynb).

#### Versions of the used packages:
- pandas: 0.24.2
- numpy: 1.16.4
- folium: 0.11.0
- ipywidgets: 7.5.1

In [1]:
import pandas as pd
from ast import literal_eval
import folium
import numpy as np
import ipywidgets as pyw

In [2]:
centroidBounds = [[[48.04821822889212, 11.67438639612782]],[[48.39922017377597, 11.75558830832081]]]
mapbox_access_token = r'MAPBOX_ACCESS_TOKEN'
tileset_ID_str = 'streets-v11'
# tileset_ID_str = 'outdoors-v11'
# tileset_ID_str = 'light-v10'
# tileset_ID_str = 'dark-v10'
# tileset_ID_str = 'satellite-v9'
# tileset_ID_str = 'satellite-streets-v11'
tilesize_pixels = '256'
#Tiles = 'OpenStreetMap'
#Tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
Tiles = f"https://api.mapbox.com/styles/v1/klavere/ckd8wc3hf06we1imiypydrqcf/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
MapboxAttribution = '<a href="https://www.mapbox.com/about/maps/">© Mapbox</a> | <a href="http://www.openstreetmap.org/about/">© OpenStreetMap</a> | <a href="https://www.mapbox.com/map-feedback/#/-74.5/40/10">Improve this map</a>'

In [3]:
def csvtodf_SC(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=';',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def csvtodf_C(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def getCoordsBack(coords):
    return literal_eval(coords);

def getCoordsListBack(coords):
    newcoordslist = literal_eval(coords)
    newcoordslist = [literal_eval(newcoordslist[0]),literal_eval(newcoordslist[1])]
    return newcoordslist;

In [4]:
celldf = csvtodf_SC('celldf')
celldf['polyCoords'] = celldf.apply(lambda row: getCoordsBack(row.polyCoords), axis = 1)
celldf['centroidCoords'] = celldf.apply(lambda row: getCoordsBack(row.centroidCoords), axis = 1)
celldf = celldf.reindex(columns = ['cellID', 'polyCoords', 'centroidCoords'])
celldf.head()

cellID                                         polyCoords  \
0       1  [[48.14605999999999, 11.510794], [48.146254999...   
1       2  [[48.23591199999999, 11.635515], [48.237070000...   
2       3  [[48.14900599999999, 11.696217], [48.153606000...   
3       4  [[48.12879600000001, 11.541556], [48.125282, 1...   
4       5  [[48.18833000000001, 11.615821], [48.193665, 1...   

                           centroidCoords  
0  [48.15396752884293, 11.51032914199828]  
1  [48.22263647015427, 11.62774480807693]  
2  [48.13564057367491, 11.70093894926043]  
3  [48.13194111905815, 11.54797042679206]  
4   [48.19669108307331, 11.6113147037113]

In [5]:
anomODdf = csvtodf_SC('anomODdf')
anomODdf['ODCoords'] = anomODdf.apply(lambda row: getCoordsListBack(row.ODCoords), axis = 1)
anomODdf = anomODdf.reindex(columns = ['startCellID', 'endCellID', 'ODCoords', 'hour',
                                       'moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus',
                                       'moves_MDD', 'privat_MDD', 'public_MDD', 'Rail_MDD', 'UBahn_MDD', 'Tram_MDD', 'Bus_MDD',
                                       'movesAnom', 'privatAnom', 'publicAnom', 'RailAnom', 'UBahnAnom', 'TramAnom', 'BusAnom'])
anomODdf.head()

startCellID  endCellID                                           ODCoords  \
0            1          1  [[48.15396752884293, 11.51032914199828], [48.1...   
1            1          1  [[48.15396752884293, 11.51032914199828], [48.1...   
2            1          1  [[48.15396752884293, 11.51032914199828], [48.1...   
3            1          1  [[48.15396752884293, 11.51032914199828], [48.1...   
4            1          1  [[48.15396752884293, 11.51032914199828], [48.1...   

   hour  moves  privat  public  Rail  UBahn  Tram  ...  UBahn_MDD  Tram_MDD  \
0     0   13.0    13.0     0.0   0.0    0.0   0.0  ...        0.0       0.0   
1     1    3.0     2.0     1.0   0.0    0.0   0.0  ...        0.0       0.0   
2     2   14.0    14.0     0.0   0.0    0.0   0.0  ...        0.0       0.0   
3     3    7.0     6.0     1.0   0.0    0.0   0.0  ...        0.0       0.0   
4     4   19.0    17.0     2.0   1.0    0.0   0.0  ...        0.0       0.0   

    Bus_MDD  movesAnom  privatAnom  publicAnom  RailAnom  UBahnAnom  TramAnom  \
0  0.666667  -1.333333   -0.666667   -0.666667       0.0        0.0       0.0   
1  0.666667  -7.000000   -7.333333    0.333333       0.0        0.0       0.0   
2  0.000000  -6.666667   -6.666667    0.000000       0.0        0.0       0.0   
3  0.000000  -1.333333   -2.333333    1.000000       0.0        0.0       0.0   
4  0.333333  -1.333333   -3.000000    1.666667       1.0        0.0       0.0   

    BusAnom  
0 -0.666667  
1  0.333333  
2  0.000000  
3  1.000000  
4  0.666667  

[5 rows x 25 columns]

## map OD anomalies

In [6]:
modes = ['moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus']
modesAnom = ['']*7
for i in range(0,7):
    modesAnom[i] = modes[i]+'Anom'

modesMDD = ['']*7
for i in range(0,7):
    modesMDD[i] = modes[i]+'_MDD'

modesColor = {'privat':'#999999',#'#BFBFBF',
              'Rail':'#4daf4a',
              'UBahn':'#377eb8',
              'Tram':'#e41a1c',
              'Bus':'#984ea3',
              'public':'#ff7f00', # '#ff7f00',
              'moves':'#a65628'} # '#FFFFFF'}
# including ColorBrewer Colors from: https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=9
PosNegCol = {True:'#1a9641',
             False:'#404040'}

d = {'ODCoords':['first']}
# modesAggMean = dict((key, ['mean']) for key in modes+modesMDD+modesAnom)
modesAggSum = dict((key, ['sum']) for key in modes+modesMDD+modesAnom)
d.update(modesAggSum)

In [7]:
def mapODAnomalies(mode, StartCells='', EndCells='', hours=(18,24), minAnom=20, widthfactor=100, relativ=False):
    
    modeAnom = mode+'Anom'
    modeMDD = mode+'_MDD'
    
    if(StartCells): startCells = list(map(int, StartCells.split(',')))
    if(EndCells): endCells = list(map(int, EndCells.split(',')))
    
    # apply startCell and endCell filters
    #d.update(modesAggMixed)
    if(StartCells):
        if(EndCells):
            mapdf = anomODdf[(anomODdf.startCellID != anomODdf.endCellID)&
                             (anomODdf.startCellID.isin(startCells))&
                             (anomODdf.endCellID.isin(endCells))].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
            mapdf.columns = mapdf.columns.get_level_values(0)
        else:
            mapdf = anomODdf[(anomODdf.startCellID != anomODdf.endCellID)&
                             (anomODdf.startCellID.isin(startCells))# &(~anomODdf.endCellID.isin(startCells))
                            ].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
            mapdf.columns = mapdf.columns.get_level_values(0)
    elif(EndCells):
        mapdf = anomODdf[(anomODdf.startCellID != anomODdf.endCellID)&
                         (anomODdf.endCellID.isin(endCells))# &(~anomODdf.startCellID.isin(endCells))
                        ].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    else:
        mapdf = anomODdf[(anomODdf.startCellID != anomODdf.endCellID)].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)

    # sum over hours
    mapdf = mapdf[(mapdf.hour.between(hours[0]-1,hours[1],inclusive=False))]
    mapdf = mapdf.groupby(['startCellID','endCellID']).agg(d).copy().reset_index()
    mapdf.columns = mapdf.columns.get_level_values(0)

    # apply minAnom filter
    mapdf = mapdf[(abs(mapdf[modeAnom]) >= minAnom)]# &(mapdf[mode]>=5)&(mapdf[modeMDD]>=5)]
    
    # initialize map with cell polygons
    m = folium.Map(tiles = Tiles,
                   attr = MapboxAttribution)
    m.fit_bounds(centroidBounds)
    for index, row in celldf.iterrows():
        if (StartCells) and row.cellID in startCells:
            color = '#FFFFFF'
            fillop = 0.5
        elif (EndCells) and row.cellID in endCells:
            color = '#000000'
            fillop = 0.5
        else:
            color = modesColor[mode]
            fillop = 0.02
        line_i = folium.Polygon(locations=row.polyCoords,
                                color = color,#'#ff7f00',#'#FBF9F7',
                                weight = 1,
                                fill = True,
                                opacity = 0.5,
                                fill_opacity = fillop,
                                tooltip = 'Cell'+str(row.cellID)).add_to(m)    
    
    # put anomalies on map
    # M = 2*np.mean(abs(mapdf[modeAnom])) # mean for linewidths if relative=False //doesn't make sense here

    for index, row in mapdf.iterrows():
        absAnom = round(abs(row[modeAnom]))
        MDD = round(row[modeMDD])
        if (row[modeAnom] > 0):
            color = PosNegCol[True]#'#1a9641'
            side = 'more'
        else:
            color = PosNegCol[False]#'#404040'
            side = 'less'
#         if (relativ):
#             weight = (absAnom/MDD)*5
#         else:
        weight = absAnom/(widthfactor+1)
        tooltip = str(absAnom)+' moves '+side+'<br>'+str(np.round(row[mode]/(MDD+1), decimals=1))+' times the moves on a normal weekday <br>mode = '+str(mode)+'<br>from Cell '+str(row.startCellID)+' to Cell '+str(row.endCellID)
        line_i = folium.PolyLine(locations = row.ODCoords,
                                 color = color,
                                 weight = weight,
                                 opacity = 0.6,
                                 tooltip = tooltip)
        m.add_child(line_i)
    return m;

In [8]:
range_slider_h = pyw.IntRangeSlider(
    value=[0,24],
    min=0, max=24, step=1,
    description='Time of day')

range_slider_mA = pyw.IntSlider(
    value=10,
    min=10, max=100, step=5,
    description='Anom. min')

range_slider_lw = pyw.IntSlider(
    value=100,
    min=0, max=500, step=10,
    description='Width factor')

In [9]:
pyw.interact_manual(mapODAnomalies,
                    StartCells='', EndCells='',
                    mode = modes,
                    hours = range_slider_h,
                    minAnom = range_slider_mA,
                    widthfactor = range_slider_lw,
                    relativ=pyw.fixed(False))

interactive(children=(Dropdown(description='mode', options=('moves', 'privat', 'public', 'Rail', 'UBahn', 'Tra…

<function __main__.mapODAnomalies(mode, StartCells='', EndCells='', hours=(18, 24), minAnom=20, widthfactor=100, relativ=False)>